<a href="https://colab.research.google.com/github/auliaoctv/4212211035_aulia_aas_machine_vision/blob/main/4212211035_aas_machine_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import torch
import numpy as np
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from tqdm import tqdm

In [8]:
pTrain = pd.read_csv('/content/emnist-mnist-train.csv', header=None, nrows=200)
pTest = pd.read_csv('/content/emnist-mnist-test.csv', header=None, nrows=200)

In [5]:
def pSourceImage(data):
      data = np.clip(data, 0, 255).astype(np.uint8).reshape(28, 28)
      return Image.fromarray(data).convert("RGB")

In [6]:
class CustomDataset(Dataset):
  def __init__ (self, dataframe, transform=None):
    self.dataframe = dataframe
    self.transform = transform
  def __len__(self):
    return len(self.dataframe)
  def __getitem__(self, idx):
    label = self.dataframe.iloc[idx, 0]
    img_data = self.dataframe.iloc[idx, 1:].values
    image = pSourceImage(img_data)
    if self.transform:
      image = self.transform(image)
    return image, label
  transform = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor()
  ])


In [7]:
pDatasetTrain = CustomDataset(pTrain, transform=CustomDataset.transform)
pDatasetVal = CustomDataset(pTest, transform=CustomDataset.transform)

pLoaderTrain = DataLoader(pDatasetTrain, batch_size=4, shuffle=True) # Training data loader
pLoaderVal = DataLoader(pDatasetVal, batch_size=4, shuffle=False) # Validation data loader

In [ ]:
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, 200)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for param in model.features.parameters():
  param.requires_grad = False

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score

pTrainArray = pTrain.to_numpy()

pAllPreds, pAllLabels = [], []

In [ ]:
print("Leave-One-Out Cross Validation Started...")

loo = LeaveOneOut()
for pIndexTrain, pIndexTest in tqdm(loo.split(pTrainArray)):

  pSampleTrain = pTrainArray[pIndexTrain]
  pSampleTest = pTrainArray[pIndexTest]

  pDatasetTrain = CustomDataset(pd.DataFrame(pSampleTrain), transform=CustomDataset.transform)
  pDatasetTest = CustomDataset(pd.DataFrame(pSampleTest), transform=CustomDataset.transform)

  pLoaderTrain = DataLoader(pDatasetTrain, batch_size=4, shuffle=True)
  pLoaderTest = DataLoader(pDatasetTest, batch_size=1, shuffle=False)

  model = models.alexnet(pretrained=True)
  model.classifier[6] = nn.Linear(4096, 200)
  model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  model.train()
  for inputs, labels in pLoaderTrain:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.no_grad():
    for inputs, labels in pLoaderTest:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      pAllPreds.append(torch.argmax(outputs, dim=1).cpu().item())
      pAllLabels.append(labels.cpu().item())

Leave-One-Out Cross Validation Started...


13it [40:57, 189.05s/it]


KeyboardInterrupt: 

In [ ]:
p_conf_matrix  = confusion_matrix(pAllLabels, pAllPreds)
p_accuracy  = accuracy_score(pAllLabels, pAllPreds)
p_precision  = precision_score(pAllLabels, pAllPreds, average="macro", zero_division=0)
p_f1 = f1_score(pAllLabels, pAllPreds, average="macro")

In [ ]:
print("\nEvaluation Results:")
print(f"Confusion Matrix:\n{p_conf_matrix }")
print(f"Accuracy: {p_accuracy :.4f}")
print(f"Precision: {p_precision :.4f}")
print(f"F1-Score: {p_f1:.4f}")